In [837]:
import pandas as pd
import altair as alt
import math
import numpy as np 

import coffee.color as c
import coffee.palletts as p

In [838]:
xls = pd.ExcelFile(
    "./data/Data viz.xlsx"
)
market_beta = pd.read_excel( xls, "Market Beta" )
factors = pd.read_excel( xls, "Factors" )
target_maturity = pd.read_excel( xls, "Target Maturity" )

In [839]:
def return_legend(title, pallett=p.green_tea, ):
    return alt.Legend(
        cornerRadius           =10,
        padding             =10,
        fillColor           =pallett.back_HEX,

        gradientStrokeWidth =2,
        gradientStrokeColor =pallett.text_HEX,

        labelColor          =pallett.text_HEX,
        labelFont           ="LuloCleanOne-Bold",
        labelFontSize       =10,

        title               =title,      
        titleColor          =pallett.text_HEX,
        titleFont           ="LuloCleanOne-Bold",
        titleFontSize       =15, 
    )

def return_axis(title, pallett=p.green_tea, ):
    return alt.Axis(
        labelColor          =pallett.text_HEX,
        labelFont           ="LuloCleanOne-Bold",
        labelFontSize       =10,

        title               =title,      
        titlePadding        =10,
        titleColor          =pallett.text_HEX,
        titleFont           ="LuloCleanOne-Bold",
        titleFontSize       =15, 
    )

def apply_config(graph, size=(300, 300), title=""):
    graph = graph.properties(
        title=title,
        width = size[0],
        height = size[1]
    )
    return graph

def apply_all( graph, pallett=p.green_tea ):
    graph = graph.configure(
        font="LuloCleanOne-Bold",
        background=pallett.back_HEX
    )
    graph = graph.configure_title(
        fontSize=30,
        color=pallett.text_HEX,
    )
    graph = graph.configure_range (
        category=pallett.grad,
        heatmap=pallett.grad
    )
    return graph

In [840]:
prim = c.color(4, 0, 117)
second = c.color(25, 17, 250)

grad = prim.return_color_grad(second, 100)

back = c.color(219, 218, 245)
text = c.color(3, 1, 64)

pallett = c.pallet(grad, prim, second, back, back, text)

In [841]:
def round_dig(x, base=10, digits=-2):
    num = x / ( math.pow( base, digits ) )
    num = math.floor(num)
    
    return ( num * ( math.pow( base, digits ) ) )

def convert_perc(perc):
    num = perc * 100
    return round_dig( num )


# **MARKET BETA**

In [842]:
def return_beta( series, title, size=(300, 300), pallett=pallett  ):

    market_beta[series] = pd.Series( map( convert_perc, market_beta[series] ))

    sorted = market_beta.copy()
    sorted = sorted.sort_values( by=series )

    return return_beta_chart( sorted, series, title, size, pallett  )

In [843]:
def return_beta_chart( data, series, title, size=(300, 300), pallett=pallett ):
    chart = alt.Chart(data)

    heatmap = chart.mark_rect().encode(
        alt.X('Size:O', 
            scale=alt.Scale(paddingInner=0),
            axis=return_axis("Market Size", pallett)),
            
        alt.Y('Geography:O', 
        scale=alt.Scale(paddingInner=0),
        axis=return_axis("Geography", pallett) ),

        color=alt.Color(series + ':Q', 
            legend=return_legend("Return (%)", pallett)
        ),)

    text = heatmap.mark_text().transform_calculate(label = '"" + datum.x + datum.y').encode(
        text=series + ':Q',
        color={'value': pallett.text_HEX})

    return apply_config((heatmap + text), size, title)

In [844]:
month = return_beta( "1 Mo", "Monthly Index Return", pallett=pallett)
year = return_beta( "1 Yr", "Monthly Index Return", pallett=pallett)

apply_all(month | year, pallett)

alt.HConcatChart(...)